<a href="https://colab.research.google.com/github/peremartra/LLMOptCost/blob/main/00/00_Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q accelerate #==0.29.3

In [2]:
!pip install -q huggingface_hub

In [3]:
import torch

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:
base_model = "bigscience/bloomz-560m"
quantized_model_name = "bloomz-560m-quantized"
#base_model = "google/gemma-2-2b"
#quantized_mode_name = "gemma-2-2b-quantized"

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device_map = {"": device} if device != "cpu" else {}

In [7]:
def clean_gpu_memory(model):
  del model
  torch.cuda.empty_cache()

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model,
                                             device_map=device_map)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

# Quantizing the model.

In [9]:
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Save the model

In [10]:
# Save the quantized model
model.save_pretrained(
    quantized_model_name,
)
tokenizer.save_pretrained(quantized_model_name)

('bloomz-560m-quantized/tokenizer_config.json',
 'bloomz-560m-quantized/special_tokens_map.json',
 'bloomz-560m-quantized/tokenizer.json')

## Upload to Hugging Face.

In [11]:
model.push_to_hub(quantized_model_name,
                  private=False,
                  use_temp_dir=False)
tokenizer.push_to_hub(quantized_model_name,
                      private=False,
                      use_temp_dir=False)

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/oopere/bloomz-560m-quantized/commit/4ee909be2855cadc27182a79d4a51859c436c444', commit_message='Upload tokenizer', commit_description='', oid='4ee909be2855cadc27182a79d4a51859c436c444', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
clean_gpu_memory(model)

In [13]:
# Reload the quantized model
quantized_model = AutoModelForCausalLM.from_pretrained(quantized_model_name,
                                                       device_map=device_map)
#quantized_model.to(device)

In [14]:
# Inference
input_text = "Tell me a joke"
input_ids = tokenizer(input_text, return_tensors="pt").to(device)

output = quantized_model.generate(**input_ids)
print(tokenizer.decode(output[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Tell me a joke about a man who is a doctor
